# Amazon Lex Create CoffeeBot

***
Copyright [2017]-[2017] Amazon.com, Inc. or its affiliates. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file except in compliance with the License. A copy of the License is located at

http://aws.amazon.com/apache2.0/

or in the "license" file accompanying this file. This file is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
***

### Prerequisites:

#### Identity and Acces Management

The user or role that executes the commands must have permissions in AWS Identity and Access Management (IAM) to perform those actions. AWS provides a set of managed policies that help you get started quickly. For our example, you should apply the following managed policy to your user or role:

    AmazonLexFullAccess

Be aware that we recommend you follow AWS IAM best practices for production implementations, which is out of scope for this workshop.

#### Coffee Bot

Use the code examples provided in this notebook to create the fictional Coffee Bot as documented in [this Github repository](https://github.com/awslabs/amz-ai-building-better-bots/blob/master/README.md).


In [ ]:
import boto3
import IPython
import base64
import time
from pprint import pprint

aws_region = 'eu-west-1'

bot = boto3.client('lex-models', region_name=aws_region)
OUTPUT_BLACKLIST = ("ResponseMetadata")

In [ ]:
slot_types = {
        'cafeBeverageType':{'values': {'mocha','latte machiato','cappucino','hot chocolate','frappucino'},
            'resolution': 'TOP_RESOLUTION'},
        'cafeBeverageSize': {'values': {'small','medium','large'},
            'resolution': 'TOP_RESOLUTION'},
        'cafeBeverageStrength':{'values': {'single','double','triple','quadruple'},
            'resolution': 'TOP_RESOLUTION'},
        'cafeCreamerType': {'values': {'skim','half and half','almond','whole'},
            'resolution': 'TOP_RESOLUTION'}}

intent_name = 'cafeOrderBeverage'
bot_name = 'CoffeeBot'

### create custom slot types

In [ ]:
for type_name in slot_types:
    all_types = bot.get_slot_types(
    nameContains=type_name
    )
    # only proceed if slot type doesn't exist yet    
    if len(all_types['slotTypes'])== 0:

        enumerations = []
        values = slot_types[type_name]['values']
        for val in values:
            enumerations.append({'value': val})

        response = bot.put_slot_type(
        name=type_name,
        enumerationValues=enumerations,
        valueSelectionStrategy=slot_types[type_name]['resolution']
        )
        if response['ResponseMetadata']['HTTPStatusCode'] ==200:
            print ("%s slot type created, checksum: %s" %(type_name,response['checksum']))
        else:
            pprint (response)
    else:
        print ("%s already exists, skipping slot type" %type_name)

### create order beverage intent

In [ ]:
response = bot.put_intent(
    name=intent_name,
    description='Handles coffee order requests',
    slots=[
        {
            'name': 'BeverageType',
            'slotConstraint': 'Required',
            'slotType': 'cafeBeverageType',
            'slotTypeVersion': '$LATEST',
            'valueElicitationPrompt': {
                'messages': [
                    {
                        'contentType': 'PlainText',
                        'content': 'What type of coffee would you like'
                    },
                    {
                        'contentType': 'PlainText',
                        'content': 'What type of coffee do you fancy?'
                    },
                ],
                'maxAttempts': 3
            },
            'priority': 1,
            'sampleUtterances': [
                'I would like to have a {BeverageType} please.','A {BeverageType} please.'
            ]
        },
        {
            'name': 'BeverageSize',
            'slotConstraint': 'Required',
            'slotType': 'cafeBeverageSize',
            'slotTypeVersion': '$LATEST',
            'valueElicitationPrompt': {
                'messages': [
                    {
                        'contentType': 'PlainText',
                        'content': 'What size would you like, small, medium or large?'
                    }
                ],
                'maxAttempts': 3
            },
            'priority': 2,
            'sampleUtterances': [
                'I have a {BeverageSize} one please.','{BeverageSize} please.'
            ]
        },
        {
            'name': 'BeverageStrength',
            'slotConstraint': 'Optional',
            'slotType': 'cafeBeverageStrength',
            'slotTypeVersion': '$LATEST',
            'valueElicitationPrompt': {
                'messages': [
                    {
                        'contentType': 'PlainText',
                        'content': 'How many shots do you want?'
                    }
                ],
                'maxAttempts': 3
            },
            'priority': 3,
            'sampleUtterances': [
                '{BeverageStrength} shot please.'
            ]
        },
        {
            'name': 'Creamer',
            'slotConstraint': 'Optional',
            'slotType': 'cafeCreamerType',
            'slotTypeVersion': '$LATEST',
            'valueElicitationPrompt': {
                'messages': [
                    {
                        'contentType': 'PlainText',
                        'content': 'What milk do you like?'
                    }
                ],
                'maxAttempts': 3
            },
            'priority': 4
        },
    ],
    sampleUtterances=[
        'I would like to order a coffee',
        'I would like to order a {BeverageType}',
        'Can I order a {BeverageType} please',
        'Can I get a {BeverageSize} {Creamer} {BeverageStrength} {BeverageType}',
        'Can I get a {BeverageType}',
        'I would like a {BeverageSize} {BeverageType}'
    ],
    followUpPrompt={
        'prompt': {
            'messages': [
                {
                    'contentType': 'PlainText',
                    'content': 'Sure. Is there anything else I can do for you?'
                }
            ],
            'maxAttempts': 2
        },
        'rejectionStatement': {
            'messages': [
                {
                    'contentType': 'PlainText',
                    'content': 'Your {BeverageSize} {BeverageType} will be ready for pick-up soon.'
                },
                {
                    'contentType': 'PlainText',
                    'content': 'We will have your {BeverageSize} {BeverageType} ready for pick-up soon.'
                },
            ]
        }
    },
    fulfillmentActivity={
        'type': 'ReturnIntent'
    }
)

if response['ResponseMetadata']['HTTPStatusCode'] ==200:
    print ('Intent created, checksum: %s' %response['checksum'])
else:
    pprint(response)

### create coffee bot

In [ ]:
response = bot.put_bot(
    name=bot_name,
    description='Coffee order bot for Amazon Lex demo',
    intents=[
        {
            'intentName': intent_name,
            'intentVersion': '$LATEST'
        },
    ],
    clarificationPrompt={
        'messages': [
            {
                'contentType': 'PlainText',
                'content': "Sorry, can you please repeat that?"
            },
            {
                'contentType': 'PlainText',
                'content': "Sorry, but I didn't understand that. Could you try again, please?"
            },
        ],
        'maxAttempts': 2
    },
    abortStatement={
        'messages': [
            {
                'contentType': 'PlainText',
                'content': 'Sorry, I could not understand. Goodbye.'
            },
        ]
    },
    idleSessionTTLInSeconds=300,
    voiceId='Salli',
    processBehavior='BUILD',
    locale='en-US',
    childDirected=False
)

if response['ResponseMetadata']['HTTPStatusCode'] ==200:
    print ('Bot created, checksum: %s' %response['checksum'])
    
    print ('Console URL: https://eu-west-1.console.aws.amazon.com/lex/home?region=eu-west-1#bot-editor:bot=%s' %bot_name)
else:
    pprint (response)

### open coffee bot in the AWS console

Open [Amazon Lex console](https://eu-west-1.console.aws.amazon.com/lex/home?region=eu-west-1#bots:) and wait for the build to complete before testing. 

### backup code for debugging and clean-up

In [ ]:
# get checksums for resource updated
response = bot.get_intent(
    name=intent_name,
    version='$LATEST')
pprint ('Intent cheksum: %s' %response['checksum'])
response = bot.get_bot(
    name=bot_name,
    versionOrAlias='$LATEST')
pprint ('Bot cheksum: %s' %response['checksum'])

In [ ]:
# delete resources

def delete_bot():
    response = bot.delete_bot(
        name=bot_name)
    if response['ResponseMetadata']['HTTPStatusCode'] ==204:
        print ("%s deleted" %bot_name)
    else:
        pprint (response)

    time.sleep(3) 

    response = bot.delete_intent(
        name=intent_name)
    if response['ResponseMetadata']['HTTPStatusCode'] ==204:
        print ("%s intent deleted" %intent_name)
    else:
        pprint (response)

    for type_name in slot_types:
        time.sleep(3) 
        response = bot.delete_slot_type(
            name=type_name
            )
        if response['ResponseMetadata']['HTTPStatusCode'] ==204:
            print ("%s slot type deleted" %type_name)
        else:
            pprint (response)
            
#delete_bot()            